# Deutsch's Algorithm

### Reference Barnett 7.2. Nielsen 1.4.3.

**Classical version:** Given an unknown function of a single bit, determine if the function is balanced or unbalanced. The balanced functions are f(x) = x and f(x) = not(x). The unbalanced functions are f(x) = 0 and f(x) = 1. Answering this question requires evaluating f(x) twice, with inputs zero and one.

**Quantum version:** The unbalanced functions are not unitary, so the quantum f(x) requires and ancillary input qbit and a second output.

In [1]:
# imports for quiskit
import numpy as np
from math import pi
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Gate
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram



The next cell creates the four oracle circuits. The upper wire has the input x which is passed unchanged to the output. The lower wire, when given |0> outputs f(x).

In [2]:
def make_oracles():
    ''' create the four quantum circuits'''
    # f(x) = not(x) 
    fnot = QuantumCircuit(2, name='fn-not')
    fnot.cx(0, 1)
    fnot.x(1)

    # f(x) = x
    fid = QuantumCircuit(2, name='fn-ident')
    fid.cx(0, 1)

    # f(x) = 0
    fzero = QuantumCircuit(2, name='fn-zero')
    fzero.i(1)

    # f(x) = 1
    fone = QuantumCircuit(2, name = 'fn-one')
    fone.x(1)
    
    return [fnot, fid, fzero, fone]


In [3]:
def make_deutsch(oracle):
    ''' embed an oracle in the Deutsch algorithm circuit'''
    deutsch = QuantumCircuit(2, 1)
    deutsch.x(1)
    deutsch.h(0)
    deutsch.h(1)
    deutsch.append(oracle.to_instruction(), [0, 1])
    deutsch.h(0)
    deutsch.measure([0], [0])
    return deutsch
    

In [8]:
# run the algorithm on all four oracles and check the results

fns = make_oracles()
print('The oracles')
for fn in fns:
    print(fn.name)
    fn.draw()
    print(fn)
    
print("\nDeutsch's algorithm run on each oracle")
for fn in fns:
    deutsch = make_deutsch(fn)
    simulator = QasmSimulator()
    compiled_circuit = transpile(deutsch, simulator)
    job = simulator.run(compiled_circuit, shots=100)
    result = job.result()
    counts = result.get_counts(deutsch)
    
    deutsch.draw()
    print(deutsch)
    print("\nTotal counts are:", counts)
    print()
    print()


The oracles
fn-not
               
q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
fn-ident
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
fn-zero
          
q_0: ─────
     ┌───┐
q_1: ┤ I ├
     └───┘
fn-one
          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

Deutsch's algorithm run on each oracle
     ┌───┐     ┌─────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├
     ├───┤┌───┐│  fn-not │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1        ├──────╫─
     └───┘└───┘└─────────┘      ║ 
c: 1/═══════════════════════════╩═
                                0 

Total counts are: {'1': 100}


     ┌───┐     ┌───────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0          ├┤ H ├┤M├
     ├───┤┌───┐│  fn-ident │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1          ├──────╫─
     └───┘└───┘└───────────┘      ║ 
c: 1/═════════════════════════════╩═
                                  0 

Total counts are: {'1': 100}


     ┌───┐     ┌──────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0         ├┤ H ├┤M├
     ├───┤┌───┐│  fn-zero │└───┘